<a href="https://colab.research.google.com/github/gabrielcdo/Pythorch/blob/main/TripletMarginLossDoggos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-metric-learning
!pip install faiss-gpu
!pip install resnet_pytorch
!pip install split_folders


     |████████████████████████████████| 105 kB 13.8 MB/s 
     |████████████████████████████████| 89.7 MB 5.5 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pytorch_metric_learning import losses, miners, distances, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ### 
from torchvision import datasets
import torchvision.models as models
import torchvision
import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

import splitfolders
from resnet_pytorch import ResNet 

In [ ]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ### 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        #self.fc1 = nn.Linear(9216, 128)
        self.fc1 = nn.Linear(43264, 128)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

In [ ]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ### 
def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print("Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(epoch, batch_idx, loss, mining_func.num_triplets))

In [ ]:
### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

In [ ]:
### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(test_embeddings, 
                                                train_embeddings,
                                                test_labels,
                                                train_labels,
                                                False)
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))

In [ ]:
 """#transforms.Grayscale(num_output_channels=1),
       #transforms.Resize((28,28)),   
                
        transforms.RandomAffine( 5 ,scale=[0.9,1.1]),
        transforms.ColorJitter(brightness=0.1,contrast=0.2,saturation=0.2,hue=0.1),
        #transforms.Normalize((0.1307,), (0.3081,)),
        #transforms.Resize((56,56)),
        transforms.Resize((112,112)),
        transforms.ToTensor(),        
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Resize((112,112)),
        transforms.RandomHorizontalFlip(), # randomly flip and rotate
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))"""

In [ ]:
device = torch.device("cuda")

transform = transforms.Compose([
        #transforms.Grayscale(num_output_channels=1),
        #transforms.Resize((28,28)),   
                
        transforms.RandomAffine( 5 ,scale=[0.9,1.1]),
        transforms.ColorJitter(brightness=0.1,contrast=0.2,saturation=0.2,hue=0.1),
        #transforms.Normalize((0.1307,), (0.3081,)),
        #transforms.Resize((56,56)),           
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Resize((112,112)),
        transforms.RandomHorizontalFlip(), # randomly flip and rotate
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

batch_size = 218

In [ ]:
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
model1 = torchvision.models.resnet34(pretrained=False, progress=True)
model1.eval()
if torch.cuda.is_available():
    print("Cuda available")    
    model1.to('cuda')

In [ ]:
#input_folder = "/content/drive/MyDrive/Dataset Dogs/Images"
#output = "/content/drive/MyDrive/Dataset Dogs" #where you want the split datasets saved. one will be created if none is set

#splitfolders.ratio(input_folder, output="output", seed=42, ratio=(.8, .2)) # ratio of split are in order of train/val/test. You can change to whatever you want. For train/val sets only, you could do .75, .25 for example.

In [ ]:
dataset1 = datasets.ImageFolder('/content/output/train',transform=transform)

dataset2 = datasets.ImageFolder('/content/drive/MyDrive/Models_training/doggos/train/DogFaceNet_Dataset_224_1_cc/after_4_bis',transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=64)
print(dataset1)

In [ ]:
"""dataset10 = datasets.FashionMNIST('.', train=True, download=True, transform=transform)
dataset20 = datasets.FashionMNIST('.', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=256)
print(type(dataset1))"""

In [ ]:
dataset1 = datasets.ImageFolder('/content/drive/MyDrive/Dataset Dogs/output/train',transform=transform)

dataset2 = datasets.ImageFolder('/content/drive/MyDrive/Dataset Dogs/output/val',transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=64)
print(dataset1)

In [ ]:
model2 = Net().to(device)
optimizer = optim.Adam(model2.parameters(), lr=0.01)


In [ ]:
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
model1 = torchvision.models.resnet34(pretrained=True, progress=True)
model1.eval()
if torch.cuda.is_available():
    print("Cuda available")    
    model1.to('cuda')
optimizer = optim.Adam(model1.parameters(), lr=0.01)


In [ ]:
### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low = 0)
loss_func = losses.TripletMarginLoss(margin = 0.3, distance = distance, reducer = reducer)
mining_func = miners.TripletMarginMiner(margin = 0.3, distance = distance, type_of_triplets = "hard")
accuracy_calculator = AccuracyCalculator(include = ("precision_at_1",), k = 1)

In [ ]:
num_epochs = 100
### pytorch-metric-learning stuff ###
for epoch in range(1, num_epochs+1):
    train(model1, loss_func, mining_func, device, train_loader, optimizer, epoch)
    test(dataset1, dataset2, model1, accuracy_calculator)
    

